プログラムの流れ
1. 2000年からの履歴を取得
1. 最新のFX情報を取得して、Dataframeに保存
1. データフレームの結合
1. ファイルの保存

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import datetime
import pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime, date, timedelta


class Get_data():
    start = datetime(2000, 1, 1)
    end = datetime.now()
    
    # 日経平均の取得
    #### https://ntk-lab.com/import_stock_data/
    def nikkei_download(self):
        # 日経平均
        df_nikkei=web.DataReader('^N225', 'yahoo',self.start,self.end)
        print ("配列長さ:",len(df_nikkei))
        df_nikkei=df_nikkei[['Open','Close']]
        df_nikkei = df_nikkei.reset_index()
        return df_nikkei

    
    def dow_dayplus(self,row):
        y=row['Date']

        if (row['weekday']>=0 and row['weekday']<=3):
            # アメリカが月〜木曜日であれば、アメリカ時間に+1 して、日本時間にマージする
            #　例　6月16日（月）(USA)  => 6月17日（火）（jst）
            y=row['Date']+ timedelta(days=1)

        elif row['weekday']==4:
            # 金曜日であれば、   +3
            #　例　6月19日（金）(USA)  => 6月22日（月）（jst）

            y=row['Date']+ timedelta(days=3)

        return y

    # ダウ平均の取得
    ### https://jp.investing.com/indices/us-30-historical-data
    
    
    def dow_download(self):
        # ダウ平均
        df_dow = web.DataReader('^DJI', 'yahoo',self.start,self.end)

        df_dow['dow_compare']=df_dow['Close']/df_dow['Open']
        df_dow=df_dow[['dow_compare']]
        df_dow = df_dow.reset_index()

        #print ("配列長さ:",len(df_dow))
        # ⭐️⭐️⭐️⭐️⭐️⭐️ここでダウ平均の時間を＋1日する
        df_dow['weekday']=df_dow['Date'].apply(lambda x: x.weekday())
        df_dow['Date']=df_dow.apply(self.dow_dayplus,axis=1)


        return df_dow

    
    def add_today_open(self,df_nikkei,open_value):

        last_day=pd.to_datetime(df_nikkei.tail(1)['Date'].values[0])
        one_weekday_after = last_day + relativedelta(weekday=0)
        one_weekday_after

        df_nikkei=df_nikkei.append(
            { 'Date':one_weekday_after,
                'Open':open_value, 
                'Close':"0"
            },ignore_index=True)
        return df_nikkei

    
    def main(self,today=""):
        #日経平均
        df_nikkei=self.nikkei_download()
        print("日経平均【完了】")
        
        
        if today!="":
            df_nikkei=self.add_today_open(df_nikkei,today)
            
            
            
        # ダウ平均
        df_dow=self.dow_download()
        print("ダウ平均【完了】")
        
        # 結合
        df=pd.merge(df_nikkei,df_dow,on=['Date'],how='inner')
        df['Close']=df['Close'].astype('float')
        df['Open']=df['Open'].astype('float')
        df=df[['Date','Open','Close','dow_compare']]
        print("結合【完了】")
        
        # ファイルの保存 
        df.to_pickle('data/stock.pkl')
        print("保存【完了】")
        display(df.tail(3))
        
        return df

In [2]:
g=Get_data()
g.main("")

配列長さ: 5118
日経平均【完了】
ダウ平均【完了】
結合【完了】
保存【完了】


,Date,Open,Close,dow_compare
4943,2020-11-18,25860.550781,25728.140625,0.999436
4944,2020-11-19,25628.730469,25634.339844,0.985373
4945,2020-11-20,25486.830078,25527.369141,1.003824


,Date,Open,Close,dow_compare
0,2000-01-04,18937.449219,19002.859375,0.987451
1,2000-01-05,19003.509766,18542.550781,0.969002
2,2000-01-06,18574.009766,18168.269531,1.012128
3,2000-01-07,18194.050781,18193.410156,1.012588
4,2000-01-11,18246.099609,18850.919922,1.003444
...,...,...,...,...
4941,2020-11-16,25652.689453,25906.929688,1.009448
4942,2020-11-17,26043.449219,26014.619141,1.009372
4943,2020-11-18,25860.550781,25728.140625,0.999436
4944,2020-11-19,25628.730469,25634.339844,0.985373
